In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

In [ ]:
from sklearn.preprocessing import StandardScaler

# Normalizar solo variables numéricas
scaler = StandardScaler()

numerical_features = ['age', 'bmi', 'children']
numerical_data = dataset[numerical_features]

# Ajustar scaler solo en train más adelante para evitar data leakage
# Pero aquí vamos a normalizar todo para simplificar

numerical_scaled = scaler.fit_transform(numerical_data)

# One-hot encoding (igual que antes)
categorical_features = ['sex', 'smoker', 'region']
categorical_data = dataset[categorical_features]

encoder = OneHotEncoder(sparse_output=False)
categorical_encoded = encoder.fit_transform(categorical_data)

# Crear DataFrame para concatenar
encoded_columns = encoder.get_feature_names_out(categorical_features)
categorical_encoded_df = pd.DataFrame(categorical_encoded, columns=encoded_columns)

# Concatenar features escaladas y categóricas codificadas
processed_dataset = pd.concat(
    [pd.DataFrame(numerical_scaled, columns=numerical_features), categorical_encoded_df],
    axis=1
)

labels = dataset['expenses'].values

# Dividir en train/test
from sklearn.model_selection import train_test_split
train_dataset, test_dataset, train_labels, test_labels = train_test_split(
    processed_dataset.values,
    labels,
    test_size=0.2,
    random_state=42
)


In [ ]:
from tensorflow.keras import layers, models, callbacks, optimizers

model = models.Sequential([
    layers.Dense(128, activation='relu', input_shape=[train_dataset.shape[1]]),
    layers.Dropout(0.2),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.1),
    layers.Dense(1)
])

model.compile(
    optimizer=optimizers.Adam(learning_rate=0.001),
    loss='mse',
    metrics=['mae', 'mse']
)

early_stop = callbacks.EarlyStopping(monitor='val_loss', patience=10)

history = model.fit(
    train_dataset, train_labels,
    validation_split=0.2,
    epochs=200,
    callbacks=[early_stop],
    verbose=2
)


In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
